In [13]:
import os
import json
from typing import List, Dict, Any, Tuple, Optional
from contextlib import redirect_stdout

# =========================
# Config
# =========================
LETTERS = ("A", "B", "C", "D")

EVAL_PATHS = [
    r"/media/miaoen/ad4277ac-5cfe-47b0-a2cc-f9e50e0da444/LLM/PrimeKG/eval_results_llama2_paperRAG/llama2_scoring_20251222_172524.jsonl",
]

LIMIT = None  # None=全量；例如 100 就只评测前100条
SAVE_REPORT_TXT = True
REPORT_PATH = "eval_report_llama_D.txt"


# =========================
# IO
# =========================
def load_jsonl(path: str) -> List[Dict[str, Any]]:
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            rows.append(json.loads(line))
    return rows


def pick_first(rec: Dict[str, Any], keys: List[str], default=None):
    for k in keys:
        if k in rec and rec[k] is not None:
            return rec[k]
    return default


def normalize_choice(x) -> str:
    """
    尽量把输出归一到 A/B/C/D。
    兼容: '1/2/3/4', 'A.', 'A)', 'Answer: A', '... option B ...' 等。
    """
    if x is None:
        return ""
    s = str(x).strip().upper()
    if not s:
        return ""

    mp = {"1": "A", "2": "B", "3": "C", "4": "D"}
    if s in mp:
        return mp[s]
    if s in LETTERS:
        return s

    # 常见前缀
    # A. / A) / A: / (A)
    for c in LETTERS:
        if s.startswith(c + ".") or s.startswith(c + ")") or s.startswith(c + ":"):
            return c
        if s.startswith("(" + c + ")"):
            return c

    # "Answer: A" / "Final: B" / "Option C"
    for c in LETTERS:
        if f"ANSWER: {c}" in s or f"FINAL: {c}" in s or f"OPTION {c}" in s:
            return c

    # 兜底：找独立字母（避免把 "ABC" 误判）
    # 用最朴素方式：用空白/标点包围
    seps = [" ", "\n", "\t", ",", ".", ";", ":", ")", "(", "[", "]", "{", "}", "|"]
    for c in LETTERS:
        for left in seps:
            for right in seps:
                if f"{left}{c}{right}" in s:
                    return c

    # 最后：如果字符串以 A/B/C/D 开头，直接取
    if len(s) >= 1 and s[0] in LETTERS:
        return s[0]

    return ""


def build_prompt_from_record(rec: Dict[str, Any]) -> str:
    """
    拼一个用于debug展示的prompt（不是用于推理）
    """
    q = pick_first(rec, ["question", "q", "query", "prompt", "input"], "")
    ctx = pick_first(rec, ["rag_context", "context", "retrieved_facts", "rag_context_preview"], "")
    out = [f"Question: {q}".strip()]
    if isinstance(ctx, str) and ctx.strip():
        out.append("\nRAG Context:\n" + ctx.strip())
    return "\n".join(out).strip()


# =========================
# Metrics
# =========================
def compute_confusion_and_metrics(
    preds: List[str],
    gold: List[str],
    prompts: List[str],
    title: str
) -> float:
    labels = list(LETTERS)
    L = len(labels)
    idx = {c: i for i, c in enumerate(labels)}
    conf = [[0] * L for _ in range(L)]

    correct_samples, wrong_samples = [], []

    for i in range(len(preds)):
        p, g = preds[i], gold[i]
        if p not in idx or g not in idx:
            continue
        conf[idx[g]][idx[p]] += 1

        if p == g:
            if len(correct_samples) < 5:
                correct_samples.append((i, p, g, prompts[i]))
        else:
            if len(wrong_samples) < 5:
                wrong_samples.append((i, p, g, prompts[i]))

    gold_cnt = {c: sum(conf[idx[c]]) for c in labels}
    pred_cnt = {c: sum(conf[r][idx[c]] for r in range(L)) for c in labels}

    def safe(a, b):
        return a / b if b > 0 else 0.0

    by_cls = {}
    for c in labels:
        i = idx[c]
        TP = conf[i][i]
        FP = pred_cnt[c] - TP
        FN = gold_cnt[c] - TP
        prec = safe(TP, TP + FP)
        rec = safe(TP, TP + FN)
        f1 = safe(2 * prec * rec, prec + rec) if (prec + rec) > 0 else 0.0
        by_cls[c] = dict(
            precision=prec, recall=rec, f1=f1,
            support=gold_cnt[c], correct=TP
        )

    total = sum(gold_cnt.values())
    correct_total = sum(conf[i][i] for i in range(L))
    acc = safe(correct_total, total)

    macro_p = sum(by_cls[c]["precision"] for c in labels) / L
    macro_r = sum(by_cls[c]["recall"] for c in labels) / L
    macro_f = sum(by_cls[c]["f1"] for c in labels) / L

    weighted_p = safe(sum(by_cls[c]["precision"] * by_cls[c]["support"] for c in labels), total)
    weighted_r = safe(sum(by_cls[c]["recall"] * by_cls[c]["support"] for c in labels), total)
    weighted_f = safe(sum(by_cls[c]["f1"] * by_cls[c]["support"] for c in labels), total)

    micro_p = acc
    micro_r = acc
    micro_f = acc

    print("\n" + "=" * 60)
    print(title)
    print("=" * 60)

    print("\n=== Distribution ===")
    print("Pred count:", pred_cnt)
    print("Gold count:", gold_cnt)

    print("\n=== Per-class ===")
    print("Class  |  Prec   Recall   F1     Support   Correct/Gold")
    for c in labels:
        d = by_cls[c]
        print(
            f"  {c}    |  {d['precision']:.4f}  {d['recall']:.4f}  {d['f1']:.4f}"
            f"    {d['support']:5d}     {d['correct']}/{d['support']}"
        )

    print("\n=== Averages ===")
    print(f"Macro-Avg  precision={macro_p:.4f}  recall={macro_r:.4f}  F1={macro_f:.4f}")
    print(f"Weighted   precision={weighted_p:.4f}  recall={weighted_r:.4f}  F1={weighted_f:.4f}")
    print(f"Micro-Avg  precision={micro_p:.4f}  recall={micro_r:.4f}  F1={micro_f:.4f}")
    print(f"ACC        = {acc:.4f}")

    return acc


def fix_hurt(base_preds: List[str], rag_preds: List[str], gold: List[str]) -> Tuple[int, int, int]:
    fix = hurt = same = 0
    for bp, rp, g in zip(base_preds, rag_preds, gold):
        if not (bp and rp and g):
            continue
        if bp != g and rp == g:
            fix += 1
        elif bp == g and rp != g:
            hurt += 1
        else:
            same += 1
    return fix, hurt, same


# =========================
# Main
# =========================
def main():
    all_records: List[Dict[str, Any]] = []
    for p in EVAL_PATHS:
        recs = load_jsonl(p)
        print(f"[Info] Loaded {len(recs)} from {p}")
        all_records.extend(recs)

    if LIMIT is not None:
        all_records = all_records[:LIMIT]

    gold: List[str] = []
    base_preds: List[str] = []
    rag_preds: List[str] = []
    prompts: List[str] = []
    skipped = 0

    for rec in all_records:
        g = normalize_choice(pick_first(rec, ["ground_truth", "gt", "answer", "label", "gold"], None))
        bp = normalize_choice(pick_first(rec, ["base_prediction", "base_pred", "pred_no_rag", "prediction", "pred"], None))
        rp = normalize_choice(pick_first(rec, ["rag_prediction", "rag_pred", "pred_with_rag"], None))

        if not g:
            skipped += 1
            continue

        # 同时评测 base + rag：两边都必须存在
        if not bp or not rp:
            skipped += 1
            continue

        gold.append(g)
        base_preds.append(bp)
        rag_preds.append(rp)
        prompts.append(build_prompt_from_record(rec))

    print(f"[Info] used={len(gold)} | skipped={skipped}")

    # 1) Base metrics
    base_acc = compute_confusion_and_metrics(base_preds, gold, prompts, title="BASE (No RAG)")

    # 2) RAG metrics
    rag_acc = compute_confusion_and_metrics(rag_preds, gold, prompts, title="RAG")

    # 3) Fix / Hurt
    fix, hurt, same = fix_hurt(base_preds, rag_preds, gold)
    print("\n" + "=" * 60)
    print("Fix / Hurt")
    print("=" * 60)
    print(f"RAG Fix  (Base错→RAG对): {fix}")
    print(f"RAG Hurt (Base对→RAG错): {hurt}")
    print(f"Same (都对/都错):       {same}")
    print(f"\nBase ACC: {base_acc:.4f} | RAG ACC: {rag_acc:.4f} | Gain: {rag_acc - base_acc:+.4f}")

    return base_acc, rag_acc


# ================
# Run (+ optional save report to txt to avoid Jupyter truncation)
# ================
if SAVE_REPORT_TXT:
    with open(REPORT_PATH, "w", encoding="utf-8") as f, redirect_stdout(f):
        main()
    print(f"[Done] Report saved to: {REPORT_PATH}")
else:
    main()


[Done] Report saved to: eval_report_llama_D.txt


In [ ]:
import os
import re
import sys
import pandas as pd
from typing import List, Dict, Tuple, Optional

# ============================================================
# Config
# ============================================================
EXP_LABEL = "A. gpt"
LETTERS = ("A", "B", "C", "D")

CSV_PATHS = [
    r"PrimeKG/results/rag_eval_20251212_175332.csv",
]

LIMIT = None  # None=全量；或填整数只取前N行

SAVE_TXT = True
OUT_TXT = "eval_report_summary_from_csv.txt"
PRINT_TO_SCREEN = False  # notebook 不想折叠就 False

# 候选列名（弱匹配：忽略大小写/下划线/空格/破折号）
CAND_GT   = ["ground_truth", "gt", "label", "answer", "gold"]
CAND_BASE = ["base_prediction", "base_pred", "pred_no_rag", "prediction", "pred", "base"]
CAND_RAG  = ["rag_prediction", "rag_pred", "pred_with_rag", "pred_rag", "rag"]

# ============================================================
# Helpers
# ============================================================
def canon(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", str(s).lower().strip())

def pick_col_fuzzy(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    mp = {canon(c): c for c in df.columns}
    for cand in candidates:
        key = canon(cand)
        if key in mp:
            return mp[key]
    return None

def normalize_choice(x) -> str:
    if x is None:
        return ""
    s = str(x).strip().upper()
    mp = {"1": "A", "2": "B", "3": "C", "4": "D"}
    if s in mp:
        return mp[s]
    if s in LETTERS:
        return s
    m = re.search(r"\b([ABCD])\b", s)
    if m:
        return m.group(1)
    if len(s) > 0 and s[0] in LETTERS:
        return s[0]
    return ""

# ============================================================
# Metrics
# ============================================================
def compute_report(preds: List[str], gold: List[str]) -> Dict:
    labels = list(LETTERS)
    L = len(labels)
    idx = {c: i for i, c in enumerate(labels)}
    conf = [[0] * L for _ in range(L)]  # rows=gold, cols=pred

    total = 0
    format_errors = 0

    for p, g in zip(preds, gold):
        if g not in idx:
            continue
        total += 1
        if p not in idx:
            format_errors += 1
            continue
        conf[idx[g]][idx[p]] += 1

    gold_cnt = {c: 0 for c in labels}
    for g in gold:
        if g in idx:
            gold_cnt[g] += 1

    pred_cnt = {c: sum(conf[r][idx[c]] for r in range(L)) for c in labels}

    def safe(a, b):
        return a / b if b > 0 else 0.0

    by_cls = {}
    for c in labels:
        i = idx[c]
        TP = conf[i][i]
        FP = pred_cnt[c] - TP
        FN = gold_cnt[c] - TP
        prec = safe(TP, TP + FP)
        rec  = safe(TP, TP + FN)
        f1   = safe(2 * prec * rec, prec + rec) if (prec + rec) > 0 else 0.0
        by_cls[c] = dict(precision=prec, recall=rec, f1=f1, support=gold_cnt[c], correct=TP)

    correct_total = sum(conf[i][i] for i in range(L))
    acc = safe(correct_total, total)

    macro_p = sum(by_cls[c]["precision"] for c in labels) / L
    macro_r = sum(by_cls[c]["recall"] for c in labels) / L
    macro_f = sum(by_cls[c]["f1"] for c in labels) / L

    weighted_p = safe(sum(by_cls[c]["precision"] * by_cls[c]["support"] for c in labels), sum(gold_cnt.values()))
    weighted_r = safe(sum(by_cls[c]["recall"]    * by_cls[c]["support"] for c in labels), sum(gold_cnt.values()))
    weighted_f = safe(sum(by_cls[c]["f1"]        * by_cls[c]["support"] for c in labels), sum(gold_cnt.values()))

    micro_p = acc
    micro_r = acc
    micro_f = acc

    return {
        "labels": labels,
        "conf": conf,                  # ✅ 新增：混淆矩阵
        "pred_cnt": pred_cnt,
        "gold_cnt": gold_cnt,
        "by_cls": by_cls,
        "macro": (macro_p, macro_r, macro_f),
        "weighted": (weighted_p, weighted_r, weighted_f),
        "micro": (micro_p, micro_r, micro_f),
        "acc": acc,
        "format_errors": format_errors,
        "total": total,
    }


def print_block(title: str, report: Dict):
    bar = "=" * 60
    print(bar)
    print(title)
    print(bar)

    print("\n=== Distribution ===")
    print("Pred count:", report["pred_cnt"])
    print("Gold count:", report["gold_cnt"])

    print("\n=== Confusion Matrix (rows=Gold, cols=Pred) ===")
    labels = report["labels"]
    conf = report["conf"]

    header = "    " + " ".join([f"{c:>5s}" for c in labels])
    print(header)
    for i, g in enumerate(labels):
        row = " ".join([f"{conf[i][j]:5d}" for j in range(len(labels))])
        print(f"{g} | {row}")

    print("\n=== Per-class ===")
    print("Class  |  Prec   Recall   F1     Support   Correct/Gold")
    for c in LETTERS:
        d = report["by_cls"][c]
        print(f"  {c}    |  {d['precision']:.4f}  {d['recall']:.4f}  {d['f1']:.4f}     {d['support']:5d}     {d['correct']}/{d['support']}")

    macro_p, macro_r, macro_f = report["macro"]
    w_p, w_r, w_f = report["weighted"]
    mi_p, mi_r, mi_f = report["micro"]

    print("\n=== Averages ===")
    print(f"Macro-Avg  precision={macro_p:.4f}  recall={macro_r:.4f}  F1={macro_f:.4f}")
    print(f"Weighted   precision={w_p:.4f}  recall={w_r:.4f}  F1={w_f:.4f}")
    print(f"Micro-Avg  precision={mi_p:.4f}  recall={mi_r:.4f}  F1={mi_f:.4f}")
    print(f"ACC        = {report['acc']:.4f}")
    print(f"format_errors = {report['format_errors']} / total_eval = {report['total']}\n")


def fix_hurt(base_preds: List[str], rag_preds: List[str], gold: List[str]) -> Tuple[int, int, int]:
    fix = hurt = same = 0
    for bp, rp, g in zip(base_preds, rag_preds, gold):
        if bp != g and rp == g:
            fix += 1
        elif bp == g and rp != g:
            hurt += 1
        else:
            same += 1
    return fix, hurt, same

# ============================================================
# Main logic
# ============================================================
def run_one_csv(csv_path: str):
    if not os.path.exists(csv_path):
        raise FileNotFoundError(csv_path)

    df = pd.read_csv(csv_path)
    if LIMIT is not None:
        df = df.head(LIMIT)

    col_gt   = pick_col_fuzzy(df, CAND_GT)
    col_base = pick_col_fuzzy(df, CAND_BASE)
    col_rag  = pick_col_fuzzy(df, CAND_RAG)

    if col_gt is None or col_base is None:
        raise KeyError(
            f"列名匹配失败（至少要有 GT 和 BASE）。\n"
            f"检测到列: {list(df.columns)}\n"
            f"匹配结果: GT={col_gt}, BASE={col_base}, RAG={col_rag}"
        )

    # ✅ 关键修复：如果找不到 RAG 列，就退化为“只有一列 pred”
    same_pred = False
    if col_rag is None:
        col_rag = col_base
        same_pred = True

    gold_all = df[col_gt].apply(normalize_choice).tolist()
    base_all = df[col_base].apply(normalize_choice).tolist()
    rag_all  = df[col_rag].apply(normalize_choice).tolist()

    # 统一评测集合：只要 gold 有效就进；pred 无效算 format error
    gold_eval, base_eval, rag_eval = [], [], []
    for g, b, r in zip(gold_all, base_all, rag_all):
        if not g:
            continue
        gold_eval.append(g)
        base_eval.append(b)
        rag_eval.append(r)

    base_report = compute_report(base_eval, gold_eval)
    rag_report  = compute_report(rag_eval,  gold_eval)

    print(EXP_LABEL)
    print(f"[Info] File: {csv_path}")
    print(f"[Info] Columns used: GT={col_gt} | BASE={col_base} | RAG={col_rag}")
    print(f"[Info] Used rows (gold valid): {len(gold_eval)}")
    if same_pred:
        print("[Warn] 这个 CSV 只有一列预测（没有 rag_pred/base_pred 分开存）。所以 BASE 和 RAG 会完全一样，Fix/Hurt 也没有意义。")
    print()

    print_block("BASE (No RAG)", base_report)
    print_block("RAG", rag_report)

    fix, hurt, same = fix_hurt(base_eval, rag_eval, gold_eval)
    print("=" * 60)
    print("Fix / Hurt")
    print("=" * 60)
    print(f"RAG Fix  (Base错→RAG对): {fix}")
    print(f"RAG Hurt (Base对→RAG错): {hurt}")
    print(f"Same (都对/都错):       {same}")
    print(f"\nBase ACC: {base_report['acc']:.4f} | RAG ACC: {rag_report['acc']:.4f} | Gain: {rag_report['acc']-base_report['acc']:+.4f}")
    print()

def main():
    for p in CSV_PATHS:
        run_one_csv(p)

# 写 txt 防止 notebook 截断
if SAVE_TXT:
    with open(OUT_TXT, "w", encoding="utf-8") as f:
        _stdout = sys.stdout
        sys.stdout = f
        try:
            main()
        finally:
            sys.stdout = _stdout
    print(f"[Done] Saved report to: {OUT_TXT}")

if PRINT_TO_SCREEN:
    main()


ParserError: Error tokenizing data. C error: Expected 9 fields in line 3, saw 10


In [ ]:
import pandas as pd
import json
import os
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def analyze_results(file_path):
    print(f"\n{'='*40}")
    print(f"正在分析文件: {file_path}")
    
    if not os.path.exists(file_path):
        print("文件不存在")
        return

    # 1. 加载文件
    try:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file_path.endswith('.jsonl'):
            data = []
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    if line.strip():
                        data.append(json.loads(line))
            df = pd.DataFrame(data)
        else:
            print("不支持的文件格式")
            return
    except Exception as e:
        print(f"读取失败: {e}")
        return

    # ==========================================
    # 2. 修正：添加你的 CSV 文件中实际的列名
    # ==========================================
    # 真实标签的可能列名
    gt_cols = [
        'Ground_Truth', 'ground_truth',  # 你的文件用的是 Ground_Truth
        'gt', 'gold', 'answer', 'final_decision', 'gt_decision', 'label'
    ]
    
    # 预测标签的可能列名 (优先找 RAG 的，如果想看 Base 的可以调整顺序)
    pred_cols = [
        'Pred_Vector_RAG',  # 你的 RAG 结果列
        'Pred_No_RAG',      # 你的 Base 结果列 (如果想评测这个，把这一行提到上面)
        'pred', 'rag_pred', 'base_pred', 'prediction', 'generated_answer'
    ]

    col_gt = next((c for c in gt_cols if c in df.columns), None)
    col_pred = next((c for c in pred_cols if c in df.columns), None)

    if not col_gt or not col_pred:
        print(f"无法识别列名。当前列: {list(df.columns)}")
        print(f"请检查 gt_cols 和 pred_cols 列表是否包含你的列名。")
        return

    print(f"识别成功: 真实值='{col_gt}', 预测值='{col_pred}'")

    # 3. 数据清洗
    df_clean = df.dropna(subset=[col_gt, col_pred]).copy()
    y_true = df_clean[col_gt].astype(str).str.strip().str.upper()
    y_pred = df_clean[col_pred].astype(str).str.strip().str.upper()
    
    valid_labels = sorted([l for l in list(set(y_true.unique()) | set(y_pred.unique())) if len(l) < 10])

    # 4.1 打印 Count
    pred_counts = y_pred.value_counts().to_dict()
    gold_counts = y_true.value_counts().to_dict()
    pred_counts_sorted = {k: pred_counts.get(k, 0) for k in valid_labels}
    gold_counts_sorted = {k: gold_counts.get(k, 0) for k in valid_labels}

    print("\n" + "="*20 + " Counts " + "="*20)
    print(f"Pred count: {pred_counts_sorted}")
    print(f"Gold count: {gold_counts_sorted}")

    # 4.2 计算混淆矩阵 & Correct
    cm = confusion_matrix(y_true, y_pred, labels=valid_labels)
    correct_counts = dict(zip(valid_labels, cm.diagonal()))
    report_dict = classification_report(y_true, y_pred, labels=valid_labels, output_dict=True, zero_division=0)

    # 4.3 打印 Per-class 表格
    print("\n" + "="*20 + " Per-class " + "="*20)
    header = f"{'Class':<6} | {'Precision':<9} | {'Recall':<9} | {'F1':<9} | {'Support':<7} | {'Correct/Gold':<12}"
    print(header)
    print("-" * len(header))

    for label in valid_labels:
        metrics = report_dict[label]
        print(f"{label:<6} | {metrics['precision']:<9.4f} | {metrics['recall']:<9.4f} | {metrics['f1-score']:<9.4f} | {int(metrics['support']):<7} | {correct_counts.get(label,0)}/{int(metrics['support'])}")

    # 4.4 打印 Averages
    print("\n" + "="*20 + " Averages " + "="*20)
    print(f"{'Averages':<12} | {'Precision':<9} | {'Recall':<9} | {'F1':<9}")
    print("-" * len(header)) # 分割线

    macro = report_dict['macro avg']
    print(f"{'Macro-Avg':<12} | {macro['precision']:<9.4f} | {macro['recall']:<9.4f} | {macro['f1-score']:<9.4f}")
    
    weighted = report_dict['weighted avg']
    print(f"{'Weighted':<12} | {weighted['precision']:<9.4f} | {weighted['recall']:<9.4f} | {weighted['f1-score']:<9.4f}")
    
    acc = accuracy_score(y_true, y_pred)
    print(f"{'Micro-Avg':<12} | {acc:<9.4f} | {acc:<9.4f} | {acc:<9.4f}")
    print(f"\n🏆 ACC = {acc:.4f}")

    # 4.5 打印混淆矩阵
    print("\n" + "="*20 + " Confusion Matrix " + "="*20)
    cm_df = pd.DataFrame(cm, index=[f"True {x}" for x in valid_labels], columns=[f"Pred {x}" for x in valid_labels])
    print(cm_df)

# ==========================================
# 运行
# ==========================================
file_path = "/media/miaoen/ad4277ac-5cfe-47b0-a2cc-f9e50e0da444/LLM/PrimeKG/eval_results_gpt35_rag/gpt35_rag_eval_20251229_170029.csv"
analyze_results(file_path)


正在分析文件: /media/miaoen/ad4277ac-5cfe-47b0-a2cc-f9e50e0da444/LLM/PrimeKG/eval_results_gpt35_rag/gpt35_rag_eval_20251229_170029.csv
✅ 识别成功: 真实值='Ground_Truth', 预测值='Pred_Vector_RAG'

==================== Counts ====================
Pred count: {'A': 766, 'B': 1351, 'C': 1402, 'D': 664}
Gold count: {'A': 1348, 'B': 1085, 'C': 925, 'D': 825}

==================== Per-class ====================
Class  | Precision | Recall    | F1        | Support | Correct/Gold
-------------------------------------------------------------------
A      | 0.6044    | 0.3435    | 0.4380    | 1348    | 463/1348
B      | 0.4271    | 0.5318    | 0.4737    | 1085    | 577/1085
C      | 0.3894    | 0.5903    | 0.4693    | 925     | 546/925
D      | 0.4398    | 0.3539    | 0.3922    | 825     | 292/825

==================== Averages ====================
Averages     | Precision | Recall    | F1       
-------------------------------------------------------------------
Macro-Avg    | 0.4652    | 0.4549    | 0.4433   

In [21]:
import pandas as pd
import json
import os
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support

def analyze_results(file_path):
    print(f"\n{'='*40}")
    print(f"正在分析文件: {file_path}")
    
    if not os.path.exists(file_path):
        print("❌ 文件不存在")
        return

    # 1. 加载文件
    try:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file_path.endswith('.jsonl'):
            data = []
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    if line.strip():
                        data.append(json.loads(line))
            df = pd.DataFrame(data)
        else:
            print("❌ 不支持的文件格式")
            return
    except Exception as e:
        print(f"❌ 读取失败: {e}")
        return

    # 2. 自动识别列名
    gt_cols = ['gt', 'gold', 'ground_truth', 'answer', 'final_decision', 'gt_decision', 'label']
    pred_cols = ['pred', 'rag_pred', 'base_pred', 'prediction', 'generated_answer']

    col_gt = next((c for c in gt_cols if c in df.columns), None)
    col_pred = next((c for c in pred_cols if c in df.columns), None)

    if not col_gt or not col_pred:
        print(f"❌ 无法识别列名。当前列: {list(df.columns)}")
        return

    # 3. 数据清洗
    df_clean = df.dropna(subset=[col_gt, col_pred]).copy()
    y_true = df_clean[col_gt].astype(str).str.strip().str.upper()
    y_pred = df_clean[col_pred].astype(str).str.strip().str.upper()
    
    # 自动确定有效标签 (排除 nan 等)
    valid_labels = sorted([l for l in list(set(y_true.unique()) | set(y_pred.unique())) if len(l) < 10])

    # ==========================================
    # 👇 这里是修改的核心部分，模仿图片的输出格式
    # ==========================================

    # 4.1 打印 Count (Pred count & Gold count)
    # 使用 value_counts() 并转为字典
    pred_counts = y_pred.value_counts().to_dict()
    gold_counts = y_true.value_counts().to_dict()
    
    # 按标签排序让显示更好看
    pred_counts_sorted = {k: pred_counts.get(k, 0) for k in valid_labels}
    gold_counts_sorted = {k: gold_counts.get(k, 0) for k in valid_labels}

    print("\n" + "="*20 + " Counts " + "="*20)
    print(f"Pred count: {pred_counts_sorted}")
    print(f"Gold count: {gold_counts_sorted}")

    # 4.2 计算混淆矩阵 (为了获取 Correct 数量)
    cm = confusion_matrix(y_true, y_pred, labels=valid_labels)
    # 对角线上的数字就是各类别预测正确的数量
    correct_counts = dict(zip(valid_labels, cm.diagonal()))

    # 4.3 获取详细指标字典
    report_dict = classification_report(y_true, y_pred, labels=valid_labels, output_dict=True, zero_division=0)

    # 4.4 打印 Per-class 表格
    print("\n" + "="*20 + " Per-class " + "="*20)
    header = f"{'Class':<6} | {'Precision':<9} | {'Recall':<9} | {'F1':<9} | {'Support':<7} | {'Correct/Gold':<12}"
    print(header)
    print("-" * len(header))

    for label in valid_labels:
        metrics = report_dict[label]
        p = metrics['precision']
        r = metrics['recall']
        f1 = metrics['f1-score']
        sup = int(metrics['support'])
        corr = correct_counts.get(label, 0)
        
        print(f"{label:<6} | {p:<9.4f} | {r:<9.4f} | {f1:<9.4f} | {sup:<7} | {corr}/{sup}")

    # 4.5 打印 Averages 表格
    print("\n" + "="*20 + " Averages " + "="*20)
    avg_header = f"{'Averages':<12} | {'Precision':<9} | {'Recall':<9} | {'F1':<9}"
    print(avg_header)
    print("-" * len(avg_header))

    # Macro Avg
    macro = report_dict['macro avg']
    print(f"{'Macro-Avg':<12} | {macro['precision']:<9.4f} | {macro['recall']:<9.4f} | {macro['f1-score']:<9.4f}")

    # Weighted Avg
    weighted = report_dict['weighted avg']
    print(f"{'Weighted':<12} | {weighted['precision']:<9.4f} | {weighted['recall']:<9.4f} | {weighted['f1-score']:<9.4f}")

    # Micro Avg (在单标签多分类中，Micro P/R/F1 都等于 Accuracy)
    acc = accuracy_score(y_true, y_pred)
    print(f"{'Micro-Avg':<12} | {acc:<9.4f} | {acc:<9.4f} | {acc:<9.4f}")
    
    print(f"\n ACC = {acc:.4f}")

    # 4.6 打印混淆矩阵
    print("\n" + "="*20 + " Confusion Matrix " + "="*20)
    cm_df = pd.DataFrame(cm, index=[f"True {x}" for x in valid_labels], columns=[f"Pred {x}" for x in valid_labels])
    print(cm_df)

# ==========================================
# 运行示例
# ==========================================
file_path_1 = "/media/miaoen/ad4277ac-5cfe-47b0-a2cc-f9e50e0da444/LLM/PrimeKG/eval_results_gpt35_rag/gpt35_rag_eval_20251229_170029.csv"
if os.path.exists(file_path_1):
    analyze_results(file_path_1)


正在分析文件: /media/miaoen/ad4277ac-5cfe-47b0-a2cc-f9e50e0da444/LLM/PrimeKG/eval_results_gpt35_rag/gpt35_rag_eval_20251229_170029.csv
❌ 无法识别列名。当前列: ['Index', 'Question', 'Ground_Truth', 'Pred_No_RAG', 'Correct_No_RAG', 'Pred_Vector_RAG', 'Correct_Vector_RAG', 'Status', 'Retrieved_Context', 'Global_Acc_No_RAG', 'Global_Acc_Vector_RAG']


Pubmedqa

In [ ]:
import os
import re
import json
import numpy as np
import pandas as pd
from typing import Dict, List, Optional, Tuple, Any


# =========================
# Config
# =========================
INPUT_PATHS = [
    # 你的结果文件（jsonl / csv 都行）
    r"/media/miaoen/ad4277ac-5cfe-47b0-a2cc-f9e50e0da444/LLM/PrimeKG/eval_results_llama2/llama2_results_165240.jsonl",
]

# 如果你的结果文件里没有 gt_decision/final_decision，就在这里填 PubMedQA parquet（用于按 id 回填 GT）
PUBMEDQA_PARQUET = r"/media/miaoen/ad4277ac-5cfe-47b0-a2cc-f9e50e0da444/LLM/data/pubmedqa_hf/pqa_labeled_splits/test.parquet"
PARQUET_ID_IS_ROW_INDEX = True  # PubMedQA 这个 parquet 通常 id=行号(0..n-1)，基本都对

LIMIT = None
OUT_DIR = "reports"
TITLE = "A. llama2 (KG.csv RAG)"

# BERTScore 配置
BERT_LANG = "en"
BERT_MODEL_TYPE = "distilbert-base-uncased"
BERT_BATCH_SIZE = 32
BERT_RESCALE_WITH_BASELINE = False


# =========================
# Helpers
# =========================
def normalize_path(p: str) -> str:
    p = p.strip()
    # 你写的 //media/... 在 Linux 有时也能用，但不建议；这里直接规整
    if p.startswith("//media/"):
        p = "/" + p.lstrip("/")
    return os.path.normpath(p)

def canon(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", str(s).lower().strip())

def pick_col(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    mp = {canon(c): c for c in df.columns}
    for cand in candidates:
        key = canon(cand)
        if key in mp:
            return mp[key]
    return None

def load_table(path: str) -> pd.DataFrame:
    path = normalize_path(path)
    if not os.path.exists(path):
        raise FileNotFoundError(path)

    ext = os.path.splitext(path)[1].lower()
    if ext == ".csv":
        df = pd.read_csv(path)
    elif ext == ".jsonl":
        rows = []
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if line:
                    rows.append(json.loads(line))
        df = pd.DataFrame(rows)
    elif ext == ".json":
        with open(path, "r", encoding="utf-8") as f:
            obj = json.load(f)
        df = pd.DataFrame(obj if isinstance(obj, list) else obj.get("data", []))
    else:
        raise ValueError(f"Unsupported file type: {ext}")

    if LIMIT is not None:
        df = df.head(LIMIT)
    return df


# =========================
# Decision parsing
# =========================
DEC_SET = ("yes", "no", "maybe")

def norm_decision(x: Any) -> str:
    if x is None:
        return ""
    s = str(x).strip().lower()
    if s in DEC_SET:
        return s
    m = re.search(r"\b(yes|no|maybe)\b", s)
    return m.group(1) if m else ""

def extract_decision_from_text(text: str) -> str:
    s = (text or "").lower()
    matches = re.findall(r"\b(yes|no|maybe)\b", s)
    return matches[-1] if matches else "unknown"


# =========================
# Decision ACC + confusion
# =========================
def compute_acc_and_confusion_decision(gt_raw: List[str], pred_raw: List[str]) -> Dict[str, Any]:
    total = 0
    support = 0
    format_errors = 0
    correct = 0

    confusion = {
        "yes->yes": 0, "yes->no": 0, "yes->maybe": 0,
        "no->yes": 0,  "no->no": 0,  "no->maybe": 0,
        "maybe->yes": 0, "maybe->no": 0, "maybe->maybe": 0
    }

    for g0, p0 in zip(gt_raw, pred_raw):
        g = norm_decision(g0)
        if not g:
            continue
        total += 1

        p = norm_decision(p0)
        if not p:
            format_errors += 1
            continue

        support += 1
        confusion[f"{g}->{p}"] += 1
        if g == p:
            correct += 1

    acc = (correct / total) if total else 0.0  # format_errors 也算错
    return {
        "acc": float(acc),
        "support": int(support),
        "format_errors": int(format_errors),
        "confusion": confusion
    }


# =========================
# ROUGE-L (pure python)
# =========================
def _lcs_len(a_tokens, b_tokens):
    n, m = len(a_tokens), len(b_tokens)
    dp = [0] * (m + 1)
    for i in range(1, n + 1):
        prev = 0
        for j in range(1, m + 1):
            tmp = dp[j]
            if a_tokens[i - 1] == b_tokens[j - 1]:
                dp[j] = prev + 1
            else:
                dp[j] = max(dp[j], dp[j - 1])
            prev = tmp
    return dp[m]

def rouge_l_f1(pred: str, ref: str) -> float:
    pred = (pred or "").strip()
    ref = (ref or "").strip()
    if not pred or not ref:
        return 0.0
    pred_tokens = re.findall(r"\w+", pred.lower())
    ref_tokens  = re.findall(r"\w+", ref.lower())
    if not pred_tokens or not ref_tokens:
        return 0.0
    lcs = _lcs_len(pred_tokens, ref_tokens)
    prec = lcs / len(pred_tokens)
    rec  = lcs / len(ref_tokens)
    return 0.0 if (prec + rec) == 0 else (2 * prec * rec / (prec + rec))


# =========================
# BERTScore
# =========================
def compute_bertscore(preds: List[str], refs: List[str]) -> Tuple[Dict[str, float], Dict[str, List[float]]]:
    preds = ["" if p is None else str(p) for p in preds]
    refs  = ["" if r is None else str(r) for r in refs]

    import torch
    import evaluate

    device = "cuda" if torch.cuda.is_available() else "cpu"
    metric = evaluate.load("bertscore")
    res = metric.compute(
        predictions=preds,
        references=refs,
        lang=BERT_LANG,
        model_type=BERT_MODEL_TYPE,
        device=device,
        batch_size=BERT_BATCH_SIZE,
        rescale_with_baseline=bool(BERT_RESCALE_WITH_BASELINE),
    )

    p_list = [float(x) for x in res["precision"]]
    r_list = [float(x) for x in res["recall"]]
    f_list = [float(x) for x in res["f1"]]

    agg = {
        "precision": float(np.mean(p_list)) if p_list else 0.0,
        "recall": float(np.mean(r_list)) if r_list else 0.0,
        "f1": float(np.mean(f_list)) if f_list else 0.0,
    }
    per = {"precision": p_list, "recall": r_list, "f1": f_list}
    return agg, per


# =========================
# Optional: backfill GT from PubMedQA parquet
# =========================
def backfill_gt_from_parquet(df: pd.DataFrame, id_col: str) -> pd.DataFrame:
    pq_path = normalize_path(PUBMEDQA_PARQUET)
    if not PUBMEDQA_PARQUET or not os.path.exists(pq_path):
        return df

    pq_df = pd.read_parquet(pq_path)
    # 期望 parquet 里有 final_decision / long_answer
    if "final_decision" not in pq_df.columns:
        return df

    if PARQUET_ID_IS_ROW_INDEX:
        # id 直接当行号
        def get_decision(i):
            try:
                return str(pq_df.iloc[int(i)]["final_decision"]).strip().lower()
            except Exception:
                return ""
        df["__gt_from_parquet"] = df[id_col].apply(get_decision)
    else:
        # 如果你的 parquet 里有某个 id 列，就改这里做 merge
        return df

    # 如果原来就有 gt_decision/final_decision，就不覆盖
    if "gt_decision" not in df.columns and "final_decision" not in df.columns:
        df["gt_decision"] = df["__gt_from_parquet"]

    return df


# =========================
# Core run
# =========================
def format_section(title: str, obj: Dict) -> str:
    return f"\n=== {title}  ===\n{json.dumps(obj, indent=2, ensure_ascii=False)}\n"

def stem_tag(path: str) -> str:
    base = os.path.basename(path)
    return os.path.splitext(base)[0]

def run_one(path: str) -> Dict[str, Any]:
    df = load_table(path)

    # ----- 关键列候选 -----
    COL_ID  = pick_col(df, ["id", "i", "idx"])
    COL_REF = pick_col(df, ["reference", "ref", "long_answer", "answer"])
    COL_GT  = pick_col(df, ["gt_decision", "final_decision", "ground_truth", "gold", "label"])

    COL_BASE_TEXT = pick_col(df, ["base_text", "base_pred", "pred_no_rag", "pred", "prediction"])
    COL_RAG_TEXT  = pick_col(df, ["rag_text", "rag_pred", "pred_with_rag"])

    # 你这份 llama2 jsonl 里是 base_kw / rag_kw（预测决策）
    COL_BASE_DEC  = pick_col(df, ["base_kw", "base_decision"])
    COL_RAG_DEC   = pick_col(df, ["rag_kw", "rag_decision"])

    if COL_REF is None:
        raise KeyError(f"找不到 reference 列。当前列: {list(df.columns)}")
    if COL_BASE_TEXT is None:
        raise KeyError(f"找不到 base_pred/base_text 列。当前列: {list(df.columns)}")
    if COL_RAG_TEXT is None:
        raise KeyError(f"找不到 rag_pred/rag_text 列。当前列: {list(df.columns)}")

    # ----- 如果没有 GT decision，尝试从 parquet 回填 -----
    if COL_GT is None:
        if COL_ID is None:
            raise KeyError(
                "结果文件里没有 gt_decision/final_decision，同时也没有 id，无法回填 GT。\n"
                f"当前列: {list(df.columns)}"
            )
        df = backfill_gt_from_parquet(df, COL_ID)
        COL_GT = pick_col(df, ["gt_decision", "final_decision"])

    if COL_GT is None:
        raise KeyError(
            "仍然找不到 gt_decision/final_decision（回填失败）。\n"
            "请确认你提供了正确的 PubMedQA parquet，且包含 final_decision。"
        )

    refs = df[COL_REF].fillna("").astype(str).tolist()
    gt   = df[COL_GT].fillna("").astype(str).tolist()

    base_text = df[COL_BASE_TEXT].fillna("").astype(str).tolist()
    rag_text  = df[COL_RAG_TEXT].fillna("").astype(str).tolist()

    # 预测决策：优先用 base_kw/rag_kw；没有就从生成文本里抽
    if COL_BASE_DEC is not None:
        base_dec = df[COL_BASE_DEC].fillna("").astype(str).apply(norm_decision).tolist()
        if sum(1 for x in base_dec if x) == 0:
            base_dec = [extract_decision_from_text(t) for t in base_text]
    else:
        base_dec = [extract_decision_from_text(t) for t in base_text]

    if COL_RAG_DEC is not None:
        rag_dec = df[COL_RAG_DEC].fillna("").astype(str).apply(norm_decision).tolist()
        if sum(1 for x in rag_dec if x) == 0:
            rag_dec = [extract_decision_from_text(t) for t in rag_text]
    else:
        rag_dec = [extract_decision_from_text(t) for t in rag_text]

    # ----- Metrics -----
    base_acc = compute_acc_and_confusion_decision(gt, base_dec)
    rag_acc  = compute_acc_and_confusion_decision(gt, rag_dec)

    rouge_base_list = [rouge_l_f1(p, r) for p, r in zip(base_text, refs)]
    rouge_rag_list  = [rouge_l_f1(p, r) for p, r in zip(rag_text,  refs)]
    rouge = {
        "base_mean": float(np.mean(rouge_base_list)) if rouge_base_list else 0.0,
        "rag_mean":  float(np.mean(rouge_rag_list)) if rouge_rag_list else 0.0,
        "gain":      float(np.mean(rouge_rag_list) - np.mean(rouge_base_list)) if rouge_base_list else 0.0,
    }

    bert = {"base": None, "rag": None, "error": None}
    bert_per = {"base": None, "rag": None}
    try:
        base_bs, base_per = compute_bertscore(base_text, refs)
        rag_bs,  rag_per  = compute_bertscore(rag_text,  refs)
        bert["base"] = base_bs
        bert["rag"]  = rag_bs
        bert_per["base"] = base_per
        bert_per["rag"]  = rag_per
    except Exception as e:
        bert["error"] = str(e)

    # expanded df
    out_df = df.copy()
    out_df["gt_decision_used"] = [norm_decision(x) for x in gt]
    out_df["base_decision_used"] = base_dec
    out_df["rag_decision_used"] = rag_dec
    out_df["rougeL_base"] = rouge_base_list
    out_df["rougeL_rag"]  = rouge_rag_list

    if bert_per["base"] is not None:
        out_df["bert_f1_base"] = bert_per["base"]["f1"]
    if bert_per["rag"] is not None:
        out_df["bert_f1_rag"] = bert_per["rag"]["f1"]
    if ("bert_f1_base" in out_df.columns) and ("bert_f1_rag" in out_df.columns):
        out_df["bert_diff"] = out_df["bert_f1_rag"] - out_df["bert_f1_base"]

    info = {
        "path": normalize_path(path),
        "rows": int(len(df)),
        "columns": list(df.columns),
        "col_id": COL_ID,
        "col_ref": COL_REF,
        "col_gt_decision": COL_GT,
        "col_base_text": COL_BASE_TEXT,
        "col_rag_text": COL_RAG_TEXT,
        "col_base_dec": COL_BASE_DEC,
        "col_rag_dec": COL_RAG_DEC,
        "task_forced": "decision(yes/no/maybe)"
    }

    return {
        "info": info,
        "metrics": {
            "bertscore": bert,
            "acc": {"base": base_acc, "rag": rag_acc},
            "rougeL": rouge
        },
        "out_df": out_df
    }


def save_outputs(result: Dict[str, Any], out_dir: str, tag: str) -> Tuple[str, str, str]:
    os.makedirs(out_dir, exist_ok=True)
    out_txt  = os.path.join(out_dir, f"{tag}_report.txt")
    out_json = os.path.join(out_dir, f"{tag}_metrics.json")
    out_csv  = os.path.join(out_dir, f"{tag}_expanded.csv")

    info = result["info"]
    m = result["metrics"]

    lines = []
    lines.append(f"{TITLE}\n" + "="*60)
    lines.append(f"[Info] Loaded: {info['path']} | rows={info['rows']}")
    lines.append(f"[Info] Columns: {info['columns']}")
    lines.append(f"[Info] Task: {info['task_forced']}")
    lines.append(f"[Info] Using cols: ref={info['col_ref']} | gt={info['col_gt_decision']} | base_text={info['col_base_text']} | rag_text={info['col_rag_text']} | base_dec={info['col_base_dec']} | rag_dec={info['col_rag_dec']}")

    if m["bertscore"]["base"] is not None:
        lines.append("\n=== Base BERTScore  ===")
        lines.append(json.dumps(m["bertscore"]["base"], indent=2, ensure_ascii=False))
        lines.append("\n=== RAG  BERTScore  ===")
        lines.append(json.dumps(m["bertscore"]["rag"], indent=2, ensure_ascii=False))
    else:
        lines.append("\n[Warn] BERTScore skipped: " + str(m["bertscore"]["error"]))

    lines.append(format_section("Base ACC", m["acc"]["base"]))
    lines.append(format_section("RAG  ACC", m["acc"]["rag"]))
    lines.append(format_section("ROUGE-L (pure python)", m["rougeL"]))

    with open(out_txt, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

    with open(out_json, "w", encoding="utf-8") as f:
        json.dump(result["metrics"], f, ensure_ascii=False, indent=2)

    result["out_df"].to_csv(out_csv, index=False)

    return out_txt, out_json, out_csv


def main():
    for p in INPUT_PATHS:
        res = run_one(p)
        tag = stem_tag(p)
        out_txt, out_json, out_csv = save_outputs(res, OUT_DIR, tag)
        print(f"[Done] Saved report : {out_txt}")
        print(f"[Done] Saved metrics: {out_json}")
        print(f"[Done] Saved csv    : {out_csv}")


if __name__ == "__main__":
    main()


KeyError: "找不到 reference 列。当前列: ['i', 'question', 'gold_decision', 'pred_decision', 'gold_text', 'pred_text', 'rag_context']"

In [ ]:
import os
import re
import json
import numpy as np
import pandas as pd
from typing import Dict, List, Optional, Tuple, Any


# =========================
# Config
# =========================
INPUT_PATHS = [
    # 你的结果文件（jsonl / csv 都行）
    r"/media/miaoen/ad4277ac-5cfe-47b0-a2cc-f9e50e0da444/LLM/PrimeKG/eval_out_kgcsv_pubmedqa_gpt35/run_20251226_182026_gpt35/results.csv",
]

# 如果你的结果文件里没有 gt_decision/final_decision，就在这里填 PubMedQA parquet（用于按 id 回填 GT）
PUBMEDQA_PARQUET = r"/media/miaoen/ad4277ac-5cfe-47b0-a2cc-f9e50e0da444/LLM/data/pubmedqa_hf/pqa_labeled_splits/test.parquet"
PARQUET_ID_IS_ROW_INDEX = True  # PubMedQA 这个 parquet 通常 id=行号(0..n-1)，基本都对

LIMIT = None
OUT_DIR = "reports"
TITLE = "A. llama2 (KG.csv RAG)"

# BERTScore 配置
BERT_LANG = "en"
BERT_MODEL_TYPE = "distilbert-base-uncased"
BERT_BATCH_SIZE = 32
BERT_RESCALE_WITH_BASELINE = False


# =========================
# Helpers
# =========================
def normalize_path(p: str) -> str:
    p = p.strip()
    # 你写的 //media/... 在 Linux 有时也能用，但不建议；这里直接规整
    if p.startswith("//media/"):
        p = "/" + p.lstrip("/")
    return os.path.normpath(p)

def canon(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", str(s).lower().strip())

def pick_col(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    mp = {canon(c): c for c in df.columns}
    for cand in candidates:
        key = canon(cand)
        if key in mp:
            return mp[key]
    return None

def load_table(path: str) -> pd.DataFrame:
    path = normalize_path(path)
    if not os.path.exists(path):
        raise FileNotFoundError(path)

    ext = os.path.splitext(path)[1].lower()
    if ext == ".csv":
        df = pd.read_csv(path)
    elif ext == ".jsonl":
        rows = []
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if line:
                    rows.append(json.loads(line))
        df = pd.DataFrame(rows)
    elif ext == ".json":
        with open(path, "r", encoding="utf-8") as f:
            obj = json.load(f)
        df = pd.DataFrame(obj if isinstance(obj, list) else obj.get("data", []))
    else:
        raise ValueError(f"Unsupported file type: {ext}")

    if LIMIT is not None:
        df = df.head(LIMIT)
    return df


# =========================
# Decision parsing
# =========================
DEC_SET = ("yes", "no", "maybe")

def norm_decision(x: Any) -> str:
    if x is None:
        return ""
    s = str(x).strip().lower()
    if s in DEC_SET:
        return s
    m = re.search(r"\b(yes|no|maybe)\b", s)
    return m.group(1) if m else ""

def extract_decision_from_text(text: str) -> str:
    s = (text or "").lower()
    matches = re.findall(r"\b(yes|no|maybe)\b", s)
    return matches[-1] if matches else "unknown"


# =========================
# Decision ACC + confusion
# =========================
def compute_acc_and_confusion_decision(gt_raw: List[str], pred_raw: List[str]) -> Dict[str, Any]:
    total = 0
    support = 0
    format_errors = 0
    correct = 0

    confusion = {
        "yes->yes": 0, "yes->no": 0, "yes->maybe": 0,
        "no->yes": 0,  "no->no": 0,  "no->maybe": 0,
        "maybe->yes": 0, "maybe->no": 0, "maybe->maybe": 0
    }

    for g0, p0 in zip(gt_raw, pred_raw):
        g = norm_decision(g0)
        if not g:
            continue
        total += 1

        p = norm_decision(p0)
        if not p:
            format_errors += 1
            continue

        support += 1
        confusion[f"{g}->{p}"] += 1
        if g == p:
            correct += 1

    acc = (correct / total) if total else 0.0  # format_errors 也算错
    return {
        "acc": float(acc),
        "support": int(support),
        "format_errors": int(format_errors),
        "confusion": confusion
    }


# =========================
# ROUGE-L (pure python)
# =========================
def _lcs_len(a_tokens, b_tokens):
    n, m = len(a_tokens), len(b_tokens)
    dp = [0] * (m + 1)
    for i in range(1, n + 1):
        prev = 0
        for j in range(1, m + 1):
            tmp = dp[j]
            if a_tokens[i - 1] == b_tokens[j - 1]:
                dp[j] = prev + 1
            else:
                dp[j] = max(dp[j], dp[j - 1])
            prev = tmp
    return dp[m]

def rouge_l_f1(pred: str, ref: str) -> float:
    pred = (pred or "").strip()
    ref = (ref or "").strip()
    if not pred or not ref:
        return 0.0
    pred_tokens = re.findall(r"\w+", pred.lower())
    ref_tokens  = re.findall(r"\w+", ref.lower())
    if not pred_tokens or not ref_tokens:
        return 0.0
    lcs = _lcs_len(pred_tokens, ref_tokens)
    prec = lcs / len(pred_tokens)
    rec  = lcs / len(ref_tokens)
    return 0.0 if (prec + rec) == 0 else (2 * prec * rec / (prec + rec))


# =========================
# BERTScore
# =========================
def compute_bertscore(preds: List[str], refs: List[str]) -> Tuple[Dict[str, float], Dict[str, List[float]]]:
    preds = ["" if p is None else str(p) for p in preds]
    refs  = ["" if r is None else str(r) for r in refs]

    import torch
    import evaluate

    device = "cuda" if torch.cuda.is_available() else "cpu"
    metric = evaluate.load("bertscore")
    res = metric.compute(
        predictions=preds,
        references=refs,
        lang=BERT_LANG,
        model_type=BERT_MODEL_TYPE,
        device=device,
        batch_size=BERT_BATCH_SIZE,
        rescale_with_baseline=bool(BERT_RESCALE_WITH_BASELINE),
    )

    p_list = [float(x) for x in res["precision"]]
    r_list = [float(x) for x in res["recall"]]
    f_list = [float(x) for x in res["f1"]]

    agg = {
        "precision": float(np.mean(p_list)) if p_list else 0.0,
        "recall": float(np.mean(r_list)) if r_list else 0.0,
        "f1": float(np.mean(f_list)) if f_list else 0.0,
    }
    per = {"precision": p_list, "recall": r_list, "f1": f_list}
    return agg, per


# =========================
# Optional: backfill GT from PubMedQA parquet
# =========================
def backfill_gt_from_parquet(df: pd.DataFrame, id_col: str) -> pd.DataFrame:
    pq_path = normalize_path(PUBMEDQA_PARQUET)
    if not PUBMEDQA_PARQUET or not os.path.exists(pq_path):
        return df

    pq_df = pd.read_parquet(pq_path)
    # 期望 parquet 里有 final_decision / long_answer
    if "final_decision" not in pq_df.columns:
        return df

    if PARQUET_ID_IS_ROW_INDEX:
        # id 直接当行号
        def get_decision(i):
            try:
                return str(pq_df.iloc[int(i)]["final_decision"]).strip().lower()
            except Exception:
                return ""
        df["__gt_from_parquet"] = df[id_col].apply(get_decision)
    else:
        # 如果你的 parquet 里有某个 id 列，就改这里做 merge
        return df

    # 如果原来就有 gt_decision/final_decision，就不覆盖
    if "gt_decision" not in df.columns and "final_decision" not in df.columns:
        df["gt_decision"] = df["__gt_from_parquet"]

    return df


# =========================
# Core run
# =========================
def format_section(title: str, obj: Dict) -> str:
    return f"\n=== {title}  ===\n{json.dumps(obj, indent=2, ensure_ascii=False)}\n"

def stem_tag(path: str) -> str:
    base = os.path.basename(path)
    return os.path.splitext(base)[0]

def run_one(path: str) -> Dict[str, Any]:
    df = load_table(path)

    # ----- 关键列候选 -----
    COL_ID  = pick_col(df, ["id", "i", "idx"])
    COL_REF = pick_col(df, ["reference", "ref", "long_answer", "answer"])
    COL_GT  = pick_col(df, ["gt_decision", "final_decision", "ground_truth", "gold", "label"])

    COL_BASE_TEXT = pick_col(df, ["base_text", "base_pred", "pred_no_rag", "pred", "prediction"])
    COL_RAG_TEXT  = pick_col(df, ["rag_text", "rag_pred", "pred_with_rag"])

    # 你这份 llama2 jsonl 里是 base_kw / rag_kw（预测决策）
    COL_BASE_DEC  = pick_col(df, ["base_kw", "base_decision"])
    COL_RAG_DEC   = pick_col(df, ["rag_kw", "rag_decision"])

    if COL_REF is None:
        raise KeyError(f"找不到 reference 列。当前列: {list(df.columns)}")
    if COL_BASE_TEXT is None:
        raise KeyError(f"找不到 base_pred/base_text 列。当前列: {list(df.columns)}")
    if COL_RAG_TEXT is None:
        raise KeyError(f"找不到 rag_pred/rag_text 列。当前列: {list(df.columns)}")

    # ----- 如果没有 GT decision，尝试从 parquet 回填 -----
    if COL_GT is None:
        if COL_ID is None:
            raise KeyError(
                "结果文件里没有 gt_decision/final_decision，同时也没有 id，无法回填 GT。\n"
                f"当前列: {list(df.columns)}"
            )
        df = backfill_gt_from_parquet(df, COL_ID)
        COL_GT = pick_col(df, ["gt_decision", "final_decision"])

    if COL_GT is None:
        raise KeyError(
            "仍然找不到 gt_decision/final_decision（回填失败）。\n"
            "请确认你提供了正确的 PubMedQA parquet，且包含 final_decision。"
        )

    refs = df[COL_REF].fillna("").astype(str).tolist()
    gt   = df[COL_GT].fillna("").astype(str).tolist()

    base_text = df[COL_BASE_TEXT].fillna("").astype(str).tolist()
    rag_text  = df[COL_RAG_TEXT].fillna("").astype(str).tolist()

    # 预测决策：优先用 base_kw/rag_kw；没有就从生成文本里抽
    if COL_BASE_DEC is not None:
        base_dec = df[COL_BASE_DEC].fillna("").astype(str).apply(norm_decision).tolist()
        if sum(1 for x in base_dec if x) == 0:
            base_dec = [extract_decision_from_text(t) for t in base_text]
    else:
        base_dec = [extract_decision_from_text(t) for t in base_text]

    if COL_RAG_DEC is not None:
        rag_dec = df[COL_RAG_DEC].fillna("").astype(str).apply(norm_decision).tolist()
        if sum(1 for x in rag_dec if x) == 0:
            rag_dec = [extract_decision_from_text(t) for t in rag_text]
    else:
        rag_dec = [extract_decision_from_text(t) for t in rag_text]

    # ----- Metrics -----
    base_acc = compute_acc_and_confusion_decision(gt, base_dec)
    rag_acc  = compute_acc_and_confusion_decision(gt, rag_dec)

    rouge_base_list = [rouge_l_f1(p, r) for p, r in zip(base_text, refs)]
    rouge_rag_list  = [rouge_l_f1(p, r) for p, r in zip(rag_text,  refs)]
    rouge = {
        "base_mean": float(np.mean(rouge_base_list)) if rouge_base_list else 0.0,
        "rag_mean":  float(np.mean(rouge_rag_list)) if rouge_rag_list else 0.0,
        "gain":      float(np.mean(rouge_rag_list) - np.mean(rouge_base_list)) if rouge_base_list else 0.0,
    }

    bert = {"base": None, "rag": None, "error": None}
    bert_per = {"base": None, "rag": None}
    try:
        base_bs, base_per = compute_bertscore(base_text, refs)
        rag_bs,  rag_per  = compute_bertscore(rag_text,  refs)
        bert["base"] = base_bs
        bert["rag"]  = rag_bs
        bert_per["base"] = base_per
        bert_per["rag"]  = rag_per
    except Exception as e:
        bert["error"] = str(e)

    # expanded df
    out_df = df.copy()
    out_df["gt_decision_used"] = [norm_decision(x) for x in gt]
    out_df["base_decision_used"] = base_dec
    out_df["rag_decision_used"] = rag_dec
    out_df["rougeL_base"] = rouge_base_list
    out_df["rougeL_rag"]  = rouge_rag_list

    if bert_per["base"] is not None:
        out_df["bert_f1_base"] = bert_per["base"]["f1"]
    if bert_per["rag"] is not None:
        out_df["bert_f1_rag"] = bert_per["rag"]["f1"]
    if ("bert_f1_base" in out_df.columns) and ("bert_f1_rag" in out_df.columns):
        out_df["bert_diff"] = out_df["bert_f1_rag"] - out_df["bert_f1_base"]

    info = {
        "path": normalize_path(path),
        "rows": int(len(df)),
        "columns": list(df.columns),
        "col_id": COL_ID,
        "col_ref": COL_REF,
        "col_gt_decision": COL_GT,
        "col_base_text": COL_BASE_TEXT,
        "col_rag_text": COL_RAG_TEXT,
        "col_base_dec": COL_BASE_DEC,
        "col_rag_dec": COL_RAG_DEC,
        "task_forced": "decision(yes/no/maybe)"
    }

    return {
        "info": info,
        "metrics": {
            "bertscore": bert,
            "acc": {"base": base_acc, "rag": rag_acc},
            "rougeL": rouge
        },
        "out_df": out_df
    }


def save_outputs(result: Dict[str, Any], out_dir: str, tag: str) -> Tuple[str, str, str]:
    os.makedirs(out_dir, exist_ok=True)
    out_txt  = os.path.join(out_dir, f"{tag}_report.txt")
    out_json = os.path.join(out_dir, f"{tag}_metrics.json")
    out_csv  = os.path.join(out_dir, f"{tag}_expanded.csv")

    info = result["info"]
    m = result["metrics"]

    lines = []
    lines.append(f"{TITLE}\n" + "="*60)
    lines.append(f"[Info] Loaded: {info['path']} | rows={info['rows']}")
    lines.append(f"[Info] Columns: {info['columns']}")
    lines.append(f"[Info] Task: {info['task_forced']}")
    lines.append(f"[Info] Using cols: ref={info['col_ref']} | gt={info['col_gt_decision']} | base_text={info['col_base_text']} | rag_text={info['col_rag_text']} | base_dec={info['col_base_dec']} | rag_dec={info['col_rag_dec']}")

    if m["bertscore"]["base"] is not None:
        lines.append("\n=== Base BERTScore  ===")
        lines.append(json.dumps(m["bertscore"]["base"], indent=2, ensure_ascii=False))
        lines.append("\n=== RAG  BERTScore  ===")
        lines.append(json.dumps(m["bertscore"]["rag"], indent=2, ensure_ascii=False))
    else:
        lines.append("\n[Warn] BERTScore skipped: " + str(m["bertscore"]["error"]))

    lines.append(format_section("Base ACC", m["acc"]["base"]))
    lines.append(format_section("RAG  ACC", m["acc"]["rag"]))
    lines.append(format_section("ROUGE-L (pure python)", m["rougeL"]))

    with open(out_txt, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

    with open(out_json, "w", encoding="utf-8") as f:
        json.dump(result["metrics"], f, ensure_ascii=False, indent=2)

    result["out_df"].to_csv(out_csv, index=False)

    return out_txt, out_json, out_csv


def main():
    for p in INPUT_PATHS:
        res = run_one(p)
        tag = stem_tag(p)
        out_txt, out_json, out_csv = save_outputs(res, OUT_DIR, tag)
        print(f"[Done] Saved report : {out_txt}")
        print(f"[Done] Saved metrics: {out_json}")
        print(f"[Done] Saved csv    : {out_csv}")


if __name__ == "__main__":
    main()


KeyError: "找不到 reference 列。当前列: ['i', 'question', 'gold_decision', 'pred_decision', 'gold_text', 'pred_text', 'rag_context']"

In [ ]:
import os
import re
import json
import numpy as np
import pandas as pd
from typing import Dict, List, Optional, Tuple, Any
from datetime import datetime

# =========================
# Config
# =========================
INPUT_PATHS = [
    r"/media/miaoen/ad4277ac-5cfe-47b0-a2cc-f9e50e0da444/LLM/PrimeKG/eval_results_compare/compare_primekg_only_20251229_1538.csv",
]
LIMIT = None
OUT_DIR = "reports"
TITLE = "A. gpt"

# BERTScore 配置
BERT_LANG = "en"
BERT_MODEL_TYPE = "distilbert-base-uncased"
BERT_BATCH_SIZE = 32
BERT_RESCALE_WITH_BASELINE = False

# =========================
# Helpers: load + column pick
# =========================
def canon(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", str(s).lower().strip())

def pick_col(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    mp = {canon(c): c for c in df.columns}
    for cand in candidates:
        k = canon(cand)
        if k in mp:
            return mp[k]
    return None

def load_table(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        raise FileNotFoundError(path)
    ext = os.path.splitext(path)[1].lower()
    if ext == ".csv":
        df = pd.read_csv(path)
    elif ext == ".jsonl":
        rows = []
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if line:
                    rows.append(json.loads(line))
        df = pd.DataFrame(rows)
    elif ext == ".json":
        with open(path, "r", encoding="utf-8") as f:
            obj = json.load(f)
        if isinstance(obj, list):
            df = pd.DataFrame(obj)
        else:
            df = pd.DataFrame(obj.get("data", []))
    else:
        raise ValueError(f"Unsupported file type: {ext}")

    if LIMIT is not None:
        df = df.head(LIMIT)
    return df

# =========================
# Task inference (decision / mcqa / text_only)
# =========================
LETTERS = ("A", "B", "C", "D")
DEC_SET = ("yes", "no", "maybe")

def norm_mcqa(x: Any) -> str:
    if x is None:
        return ""
    s = str(x).strip().upper()
    mp = {"1":"A","2":"B","3":"C","4":"D"}
    if s in mp: return mp[s]
    if s in LETTERS: return s
    m = re.search(r"\b([ABCD])\b", s)
    return m.group(1) if m else ""

def norm_decision(x: Any) -> str:
    if x is None:
        return ""
    s = str(x).strip().lower()
    if s in DEC_SET: return s
    m = re.search(r"\b(yes|no|maybe)\b", s)
    return m.group(1) if m else ""

def extract_decision_from_text(text: str) -> str:
    s = (text or "").lower()
    matches = re.findall(r"\b(yes|no|maybe)\b", s)
    return matches[-1] if matches else "unknown"

def infer_task_from_gold(gold_list: List[str]) -> str:
    mc = sum(1 for g in gold_list if norm_mcqa(g))
    dc = sum(1 for g in gold_list if norm_decision(g))
    if mc == 0 and dc == 0:
        return "text_only"
    return "mcqa" if mc >= dc else "decision"

# =========================
# Metrics: ACC + confusion
# =========================
def compute_acc_and_confusion_decision(gt_raw: List[str], pred_raw: List[str]) -> Dict[str, Any]:
    total = 0
    support = 0
    format_errors = 0
    correct = 0
    confusion = {
        "yes->yes": 0, "yes->no": 0, "yes->maybe": 0,
        "no->yes": 0,  "no->no": 0,  "no->maybe": 0,
        "maybe->yes": 0, "maybe->no": 0, "maybe->maybe": 0
    }

    for g0, p0 in zip(gt_raw, pred_raw):
        g = norm_decision(g0)
        if not g:
            continue
        total += 1

        p = norm_decision(p0)
        if not p:
            format_errors += 1
            continue

        support += 1
        key = f"{g}->{p}"
        if key in confusion:
            confusion[key] += 1
        if g == p:
            correct += 1

    acc = (correct / total) if total else 0.0
    return {"acc": float(acc), "support": int(support), "format_errors": int(format_errors), "confusion": confusion}

def compute_acc_and_confusion_mcqa(gt_raw: List[str], pred_raw: List[str]) -> Dict[str, Any]:
    labels = list(LETTERS)
    idx = {c:i for i,c in enumerate(labels)}
    conf = [[0]*4 for _ in range(4)]

    total = 0
    support = 0
    format_errors = 0
    correct = 0

    for g0, p0 in zip(gt_raw, pred_raw):
        g = norm_mcqa(g0)
        if not g:
            continue
        total += 1

        p = norm_mcqa(p0)
        if p not in idx:
            format_errors += 1
            continue

        support += 1
        conf[idx[g]][idx[p]] += 1
        if g == p:
            correct += 1

    acc = (correct / total) if total else 0.0
    confusion = {f"{g}->{p}": conf[idx[g]][idx[p]] for g in labels for p in labels}
    return {"acc": float(acc), "support": int(support), "format_errors": int(format_errors), "confusion": confusion}

# =========================
# BERTScore
# =========================
def compute_bertscore(preds: List[str], refs: List[str]) -> Tuple[Dict[str, float], Dict[str, List[float]]]:
    preds = ["" if p is None else str(p) for p in preds]
    refs  = ["" if r is None else str(r) for r in refs]

    import torch
    import evaluate

    device = "cuda" if torch.cuda.is_available() else "cpu"
    metric = evaluate.load("bertscore")
    res = metric.compute(
        predictions=preds,
        references=refs,
        lang=BERT_LANG,
        model_type=BERT_MODEL_TYPE,
        device=device,
        batch_size=BERT_BATCH_SIZE,
        rescale_with_baseline=bool(BERT_RESCALE_WITH_BASELINE),
    )

    p_list = [float(x) for x in res["precision"]]
    r_list = [float(x) for x in res["recall"]]
    f_list = [float(x) for x in res["f1"]]

    agg = {
        "precision": float(np.mean(p_list)) if p_list else 0.0,
        "recall": float(np.mean(r_list)) if r_list else 0.0,
        "f1": float(np.mean(f_list)) if f_list else 0.0,
        "model_type": BERT_MODEL_TYPE,
        "rescale_with_baseline": bool(BERT_RESCALE_WITH_BASELINE),
        "device": device,
        "batch_size": int(BERT_BATCH_SIZE),
        "hashcode": res.get("hashcode"),
    }
    per = {"precision": p_list, "recall": r_list, "f1": f_list}
    return agg, per

# =========================
# Reporting / Save
# =========================
def format_section(title: str, obj: Dict) -> str:
    return f"\n=== {title}  ===\n{json.dumps(obj, indent=2, ensure_ascii=False)}\n"

def stem_tag(path: str) -> str:
    base = os.path.basename(path)
    return os.path.splitext(base)[0]

def run_one(path: str) -> Dict[str, Any]:
    df = load_table(path)
    info = {"path": path, "rows": int(len(df)), "columns": list(df.columns)}

    # --- 修复部分：添加 "gt" 到候选列表 ---
    COL_REF = pick_col(df, ["reference", "ref", "long_answer", "answer", "gold_answer", "gold", "gt"])
    COL_GT  = pick_col(df, ["gt_decision", "ground_truth", "gold", "label", "final_decision", "gt"])

    # 预测列
    COL_BASE_TEXT = pick_col(df, ["base_text", "base_pred", "base_prediction", "pred_no_rag", "pred"])
    COL_RAG_TEXT  = pick_col(df, ["rag_text", "rag_pred", "rag_prediction", "pred_with_rag", "pred_vector_rag", "pred"])

    if COL_REF is None and COL_GT is not None:
        COL_REF = COL_GT  # 没有 reference，用 gt 做 reference
    if COL_REF is None:
        raise KeyError(f"找不到 reference/ref/gold/gt 列。当前列: {list(df.columns)}")

    if COL_GT is None:
        COL_GT = pick_col(df, ["gold", "gt"]) # Double check
    if COL_GT is None:
        raise KeyError(f"找不到 GT 列（gt_decision/ground_truth/gold/gt）。当前列: {list(df.columns)}")

    if COL_BASE_TEXT is None:
        raise KeyError(f"找不到预测列 pred/base_pred。当前列: {list(df.columns)}")
    
    # 如果找不到 rag 预测列，默认用 base 列 (因为你的文件只有一列 pred)
    if COL_RAG_TEXT is None:
        COL_RAG_TEXT = COL_BASE_TEXT  

    refs = df[COL_REF].fillna("").astype(str).tolist()
    gold = df[COL_GT].fillna("").astype(str).tolist()
    base_text = df[COL_BASE_TEXT].fillna("").astype(str).tolist()
    rag_text  = df[COL_RAG_TEXT].fillna("").astype(str).tolist()

    task = infer_task_from_gold(gold)

    info.update({
        "task": task,
        "col_ref": COL_REF,
        "col_gt": COL_GT,
        "col_base_text": COL_BASE_TEXT,
        "col_rag_text": COL_RAG_TEXT,
        "same_pred_col": (COL_BASE_TEXT == COL_RAG_TEXT),
    })

    # --- BERTScore（对 base_text vs refs，rag_text vs refs）---
    bert = {"base": None, "rag": None, "error": None}
    bert_per = {"base": None, "rag": None}
    try:
        base_bs, base_per = compute_bertscore(base_text, refs)
        rag_bs, rag_per   = compute_bertscore(rag_text,  refs)
        bert["base"], bert["rag"] = base_bs, rag_bs
        bert_per["base"], bert_per["rag"] = base_per, rag_per
    except Exception as e:
        bert["error"] = str(e)

    # --- ACC ---
    acc = {"base": None, "rag": None}
    if task == "decision":
        base_dec = [extract_decision_from_text(t) for t in base_text]
        rag_dec  = [extract_decision_from_text(t) for t in rag_text]
        acc["base"] = compute_acc_and_confusion_decision(gold, base_dec)
        acc["rag"]  = compute_acc_and_confusion_decision(gold, rag_dec)
    elif task == "mcqa":
        acc["base"] = compute_acc_and_confusion_mcqa(gold, base_text)
        acc["rag"]  = compute_acc_and_confusion_mcqa(gold, rag_text)
    else:
        acc["base"] = {"skipped": "gold 既非决策也非MCQA格式"}
        acc["rag"]  = {"skipped": "gold 既非决策也非MCQA格式"}

    # --- 输出扩展表 ---
    out_df = df.copy()
    if bert_per["base"] is not None:
        out_df["bert_f1_base"] = bert_per["base"]["f1"]
    if bert_per["rag"] is not None:
        out_df["bert_f1_rag"] = bert_per["rag"]["f1"]
    if ("bert_f1_base" in out_df.columns) and ("bert_f1_rag" in out_df.columns):
        out_df["bert_diff"] = out_df["bert_f1_rag"] - out_df["bert_f1_base"]

    return {"info": info, "metrics": {"bertscore": bert, "acc": acc}, "out_df": out_df}

def save_outputs(result: Dict[str, Any], out_dir: str, tag: str) -> Tuple[str, str, str]:
    os.makedirs(out_dir, exist_ok=True)
    out_txt  = os.path.join(out_dir, f"{tag}_report.txt")
    out_json = os.path.join(out_dir, f"{tag}_metrics.json")
    out_csv  = os.path.join(out_dir, f"{tag}_expanded.csv")

    info = result["info"]
    m = result["metrics"]

    lines = []
    lines.append(f"{TITLE}\n" + "="*60)
    lines.append(f"[Info] Loaded: {info['path']} | rows={info['rows']}")
    lines.append(f"[Info] Columns: {info['columns']}")
    lines.append(f"[Info] Task: {info['task']}")
    lines.append(f"[Info] Using cols: ref={info['col_ref']} | gt={info['col_gt']} | base={info['col_base_text']} | rag={info['col_rag_text']}")
    if info.get("same_pred_col", False):
        lines.append("[Warn] 这是一个单预测文件，Base 和 RAG 指标将显示相同的值。")

    if m["bertscore"]["base"] is not None:
        lines.append("\n=== Base BERTScore  ===")
        lines.append(json.dumps(m["bertscore"]["base"], indent=2, ensure_ascii=False))
        lines.append("\n=== RAG  BERTScore  ===")
        lines.append(json.dumps(m["bertscore"]["rag"], indent=2, ensure_ascii=False))
    else:
        lines.append("\n[Warn] BERTScore skipped: " + str(m["bertscore"]["error"]))

    lines.append(format_section("Base ACC", m["acc"]["base"]))
    lines.append(format_section("RAG  ACC", m["acc"]["rag"]))

    with open(out_txt, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

    with open(out_json, "w", encoding="utf-8") as f:
        json.dump(result["metrics"], f, ensure_ascii=False, indent=2)

    result["out_df"].to_csv(out_csv, index=False)
    return out_txt, out_json, out_csv

def main():
    for p in INPUT_PATHS:
        try:
            res = run_one(p)
            tag = stem_tag(p)
            out_txt, out_json, out_csv = save_outputs(res, OUT_DIR, tag)
            print(f"[Done] Processed {p}")
            print(f"       Report : {out_txt}")
            print(f"       CSV    : {out_csv}")
        except Exception as e:
            print(f"[Error] Failed to process {p}: {e}")

if __name__ == "__main__":
    main()

[Error] Failed to process /media/miaoen/ad4277ac-5cfe-47b0-a2cc-f9e50e0da444/LLM/eval_out/pubmedqa_rag_20251222_143546.jsonl: "找不到 reference/ref/gold/gt 列。当前列: ['i', 'question', 'gold_decision', 'pred_decision', 'gold_text', 'pred_text', 'rag_context']"
